# Many simulations at different carrying capacities and switching speeds

In [414]:
import pandas as pd
import numpy as np
import math
import os
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [435]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [433]:
def hist_carrying_cap(landscape, k_values=[9, 8, 7, 6], num=100, **kargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    data = []
    for k in k_values:
        param = dict(kargs)
        param['carrying_cap'] = 10**k
        param['prob_mutation'] = 10**(-(k-1))
        times = [simulate(landscape, **param)['T_f'] for i in range(num)]
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            #xbins = dict(
            #    start = 0,
            #    end = 1000,
            #    size = 10),
            #autobinx = False,
            opacity=0.75,
            name = 'K=10^{}'.format(k)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different carrying capacities'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## Using histograms to display effect of carrying capacity

In [434]:
hist_carrying_cap(dataset2.loc['CPD'], [9, 7, 5, 3])
hist_carrying_cap(pairs[0], [9, 7, 5, 3], frequency=100)

In [164]:
def hist_switching(landscapes, frequencies=[200, 100, 50], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            xbins = dict(
                size = 1),
            autobinx = False,
            opacity=0.75,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            #rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## High switching speeds show large variation in T<sub>f</sub>

Histograms may be less useful in this case. See the box plots below.

In [436]:
hist_switching(pairs[0])
hist_switching(pairs[1])
hist_switching(pairs[2])

In [161]:
def box_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Box(
            y = times,
            boxpoints = False,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Switching speed',
        ),
        yaxis = dict(
            title = 'Time to fixation'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [409]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    T_f_sum = 0
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
            T_f_sum += results['T_f']
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {total_success_count / num}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_rate': total_success_count / num
        }

In [417]:
def table_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    success_rates = []
    weights = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, **param)
        success_rates.append(analysis['success_rate'])
        weights.append(analysis['max_weight'])
    df = pd.DataFrame({'f': frequencies, 'Success rate': success_rates, 'Weight of dominant path': weights}).set_index('f')
    return df.T

## CTX + SAM

### K=10^9

In [333]:
display(table_switching(pairs[0]))
box_switching(pairs[0])

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.0,1.0,1.0,1.0,1.0,1.0,1.00,0.79,0.08,0.0,0.0,0.0,0.0
Weight of dominant path,1.0,1.0,1.0,1.0,1.0,1.0,0.87,1.00,1.00,NaN,NaN,NaN,NaN


### K=10^6

In [334]:
display(table_switching(pairs[0], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[0], carrying_cap=10**6, prob_mutation=10**-5)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.470000,0.0,0.0
Weight of dominant path,0.87,0.92,0.89,0.86,0.93,0.94,0.96,0.53,0.85,0.93,0.914894,NaN,NaN


### K=10^4

In [402]:
display(table_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.0,1.00,0.530000,0.090000
Weight of dominant path,0.53,0.51,0.46,0.51,0.57,0.52,0.45,0.5,0.49,0.6,0.61,0.716981,0.333333


## AM + AMC

### K=10^9

In [403]:
display(table_switching(pairs[1]))
box_switching(pairs[1])

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.00,1.0,1.00,1.0,1.00,1.00,1.00,0.02,0.03,0.060000,0.01,0.02,0.0
Weight of dominant path,0.88,0.9,0.93,0.9,0.82,0.96,0.97,1.00,1.00,0.833333,1.00,0.50,NaN


### K=10^6

In [404]:
display(table_switching(pairs[1], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[1], carrying_cap=10**6, prob_mutation=10**-5)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.00,1.00,1.00,1.0,1.00,1.0,1.00,1.00,1.00,1.00,1.0,1.00,1.00
Weight of dominant path,0.68,0.67,0.69,0.8,0.73,0.6,0.93,0.93,0.94,0.95,0.9,0.93,0.88


### K=10^4

In [405]:
display(table_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0
Weight of dominant path,0.28,0.19,0.27,0.26,0.22,0.25,0.32,0.46,0.55,0.49,0.62,0.61,0.6


## ZOX + CXM

### K=10^9

In [406]:
display(table_switching(pairs[2]))
box_switching(pairs[2])

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.00,1.0,1.00,1.00,1.00,1.00
Weight of dominant path,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.99,1.0,0.93,0.99,0.98,0.98


### K=10^6

In [407]:
display(table_switching(pairs[2], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[2], carrying_cap=10**6, prob_mutation=10**-5)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
Weight of dominant path,1.0,1.0,1.0,0.97,0.98,0.97,0.99,0.91,0.68,0.95,0.85,0.88,0.82


### K=10^4

In [463]:
display(table_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3)

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00
Weight of dominant path,0.9,0.83,0.84,0.8,0.78,0.83,0.84,0.79,0.76,0.8,0.68,0.48,0.63


## Carrying capacities below 10^4 do not succeed

The tables below are for K=10^3:

In [424]:
for pair in pairs:
    display(Markdown('__{} + {}__'.format(pair[0].name, pair[1].name)))
    display(table_switching(pair, carrying_cap=10**3, prob_mutation=10**-2))

__CTX + SAM__

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02
Weight of dominant path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.50


__AM + AMC__

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Weight of dominant path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


__ZOX + CXM__

f,200,175,150,125,100,75,50,25,20,15,10,5,1
Success rate,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Weight of dominant path,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Is switching at a frequency of 1 the same as averaging 2 landscapes?

In [460]:
def avg_landscape(pair):
    return [(i + j) / 2 for i, j in zip(pair[0], pair[1])]
display(Markdown('### Average of ZOX and CXM'))
plot_simulation(simulate(avg_landscape(pairs[2])))
pathway_analysis(pd.Series(avg_landscape(pairs[2]), name='avg'), pretty=True)
display(Markdown('### Switching between ZOX and CXM at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[2]], frequency=1))
pathway_analysis(pairs[2], pretty=True, frequency=1)

### Average of ZOX and CXM

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0100,0110,0111",Yes,Yes,96,0.96
1,"0000,0010,0110,0111",Yes,No,4,0.04


Global optimum: 0111

Weight of the dominant successful pathway: 0.96

Success rate: 1.0

### Switching between ZOX and CXM at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010,0011,0111",No,Yes,0,nan
1,"0000,0100,0110,0111",Yes,No,97,0.97
2,"0000,0010,0110,0111",Yes,No,3,0.03


Global optimum: 0111

Weight of the dominant successful pathway: 0.97

Success rate: 1.0

They are pretty similar!

In [462]:
display(Markdown('### Average of AM and AMC'))
plot_simulation(simulate(avg_landscape(pairs[1]), carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pd.Series(avg_landscape(pairs[1]), name='avg'), pretty=True, carrying_cap=10**6, prob_mutation=10**-5)
display(Markdown('### Switching between AM and AMC at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[1]], frequency=1, carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pairs[1], pretty=True, frequency=1, carrying_cap=10**6, prob_mutation=10**-5)

### Average of AM and AMC

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0001,1001,1101",Yes,Yes,90,0.9
1,"0000,0001,1001,1011,1111,1101",Yes,No,5,0.05
2,"0000,0001,0101,1101",Yes,No,3,0.03
3,"0000,1000,1001,1101",Yes,No,1,0.01
4,"0000,0100,1100,1101",Yes,No,1,0.01


Global optimum: 1101

Weight of the dominant successful pathway: 0.9

Success rate: 1.0

### Switching between AM and AMC at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010",No,Yes,0,nan
1,"0000,0001,1001,1101",Yes,No,86,0.86
2,"0000,0001,0101,1101",Yes,No,9,0.09
3,"0000,1000,1001,1101",Yes,No,4,0.04
4,"0000,0001,0011,1011,1111,1101",Yes,No,1,0.01


Global optimum: 1101

Weight of the dominant successful pathway: 0.86

Success rate: 1.0